In [1]:
import sys
sys.path.append("../../")

from shared.sae_actions import load_pretrained_sae, sae_featurize_data 
from shared.models import MiniPileDataset

%load_ext autoreload
%autoreload 2




In [2]:
# Load SAE
sae = load_pretrained_sae("../training_sae/saes/spam_messages_4_20241106_180053")

featurize train

In [3]:
import pandas as pd
import numpy as np

def prepare_dataset(sentences_file, embeddings_file, sae, num_features=200, feature_registry_file="feature_registry.npy", label_column='label', text_key="text"):
    # Load dataset
    df = pd.read_csv(sentences_file)
    
    # Create MiniPileDataset
    mini_pile_dataset = MiniPileDataset(sentences_file, embeddings_file, key=text_key)
    
    # Featurize data
    try:
        X = np.memmap(
            feature_registry_file,
            dtype="float32",
            mode="r",
            shape=(sae.encoder.weight.shape[0], len(mini_pile_dataset.sentences)),
        )
    except FileNotFoundError:
        X = sae_featurize_data(mini_pile_dataset, sae, output_file=feature_registry_file)
    
    X = X.T
    
    # Apply num_features if specified
    if num_features is not None:
        X = X[:, :num_features]
    
    y = np.where(df[label_column] == 'ham', 0, 1)
    
    return X, y

In [10]:
sentences_file = "../data_preparation/data/spam_messages_train.csv"
embeddings_file = "../data_preparation/embedding_chunks/embedded_chunks/spam_messages_train_20241106_234743/embeddings.npy"
X_train, y_train = prepare_dataset(sentences_file, embeddings_file, sae, feature_registry_file="feature_registry_train_all.npy")

Featurizing data:   0%|          | 0/47392 [00:00<?, ?it/s]

In [11]:
sentences_file = "../data_preparation/data/spam_messages_val.csv"
embeddings_file = "../data_preparation/embedding_chunks/embedded_chunks/spam_messages_val_20241107_005540/embeddings.npy"
X_val, y_val = prepare_dataset(sentences_file, embeddings_file, sae, feature_registry_file="feature_registry_val_all.npy")

Featurizing data:   0%|          | 0/5923 [00:00<?, ?it/s]

In [12]:
sentences_file = "../data_preparation/data/spam_messages_test.csv"
embeddings_file = "../data_preparation/embedding_chunks/embedded_chunks/spam_messages_test_20241107_005747/embeddings.npy"
X_test, y_test = prepare_dataset(sentences_file, embeddings_file, sae, feature_registry_file="feature_registry_test_all.npy")

Featurizing data:   0%|          | 0/5926 [00:00<?, ?it/s]

feature selection

In [37]:
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy

boruta_selector = BorutaPy(
  RandomForestClassifier(n_jobs=-1, class_weight="balanced", max_depth=5),
  n_estimators="auto",
  verbose=0,
  random_state=1,
)
boruta_selector.fit(X_train[:,:200], y_train)

BorutaPy(estimator=RandomForestClassifier(class_weight='balanced', max_depth=5,
                                          n_estimators=266, n_jobs=-1,
                                          random_state=RandomState(MT19937) at 0x788729CC9F40),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x788729CC9F40)

In [39]:
boruta_selector.n_features_

79

In [40]:
boruta_selector.ranking_

array([ 56,  10,   1,   4,   1,  24,   1,  43,   2,  20, 106,   1,  14,
         1,  70,   8, 106,   1, 106,   1,   7,  67,   1,   1,   1,   1,
         1,   1,   1,  94, 106,   1,   1,   2,   1,  90, 106,   1,   1,
         2,  66,   1,  63,  42,  65,   1,  49,  55,   1,  94,   1,   1,
        10,   7,   2,  10,   1,  91,   1,   1,   1,  94,  22,   1,  37,
        74,   1,  81,   1,  94,  85,   1,  44,   1,  88,  32,  35,   1,
         1,  69,  53,   1,   2, 106,   1,   1,  40,  29,  12,   1,   1,
        27,  38,  18,   1,  39,  86,  35,   1,   1,   1, 106,   1,  55,
         1,  35,  32,   3,  72,  58,  76, 106, 106,   1,   1,   1,  32,
         1,  89,   2,  16,  22,  30,   1,  15,  64,   1,   1,  85,  50,
       106,   1,   1,   1,   1,  72, 106,   1,  75,  19, 106,   2,  51,
        73,  17, 106,  29, 106, 106,   1,   1, 106,   1,  59,   1,   1,
         1, 106,  94,  82,  47, 106,   1,   1,   2,  83,  77,   1,   1,
        41,  68,   5,   1,  57,  80,  87,  62,  61,   1,   1,  4

In [18]:
features_folder = "../feature_extraction/features/20241106_222955"

def get_feature_labels_from_mask(mask):
    import json
    import os
    from tabulate import tabulate
    
    feature_info = []
    for i, is_selected in enumerate(mask):
        if is_selected:
            feature_file = os.path.join(features_folder, f"feature_{i}.json")
            with open(feature_file) as f:
                feature_data = json.load(f)
                feature_info.append({
                    'index': i,
                    'label': feature_data['label'],
                    'confidence': feature_data.get('confidence', 'N/A')
                })
    
    # Print nicely formatted table
    headers = ['Index', 'Label', 'Confidence']
    table = [[info['index'], info['label'], info['confidence']] for info in feature_info]
    print(tabulate(table, headers=headers, tablefmt='grid'))
    
    return [info['label'] for info in feature_info]


In [9]:
print("\nSelected Features:")
get_feature_labels_from_mask(boruta_selector.support_weak_)


Selected Features:


NameError: name 'get_feature_labels_from_mask' is not defined

ok let's actually train it and evaluate it

In [15]:
from classifier_model import BinaryClassifierModel

# Initialize model
model = BinaryClassifierModel()

# Train model
model.train_model(
    X_train=X_train, 
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    use_feature_selection=False
)

# Evaluate model
model.evaluate_model(X_test, y_test)

Scaling features...
Will try 5 different max_depths and 3 different n_estimators
Total combinations to try: 15


Training Progress:   0%|          | 0/15 [00:00<?, ?it/s]


Iteration Results (max_depth=10, n_estimators=10):
Accuracy: 0.7069
Precision: 0.9684
Recall: 0.3004
F1: 0.4585
AUC-ROC: 0.6723

Iteration Results (max_depth=10, n_estimators=50):
Accuracy: 0.7138
Precision: 0.9631
Recall: 0.3196
F1: 0.4799
AUC-ROC: 0.6850

Iteration Results (max_depth=10, n_estimators=100):
Accuracy: 0.7138
Precision: 0.9619
Recall: 0.3200
F1: 0.4802
AUC-ROC: 0.6865

Iteration Results (max_depth=20, n_estimators=10):
Accuracy: 0.7199
Precision: 0.9603
Recall: 0.3359
F1: 0.4977
AUC-ROC: 0.6864

Iteration Results (max_depth=20, n_estimators=50):
Accuracy: 0.7196
Precision: 0.9613
Recall: 0.3347
F1: 0.4965
AUC-ROC: 0.6879

Iteration Results (max_depth=20, n_estimators=100):
Accuracy: 0.7189
Precision: 0.9589
Recall: 0.3339
F1: 0.4953
AUC-ROC: 0.6921

Iteration Results (max_depth=30, n_estimators=10):
Accuracy: 0.7223
Precision: 0.9546
Recall: 0.3441
F1: 0.5059
AUC-ROC: 0.6897

Iteration Results (max_depth=30, n_estimators=50):
Accuracy: 0.7229
Precision: 0.9538
Recall: 

let's try it with a logistic regression model

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np

# Initialize scaler and model
scaler = MinMaxScaler(feature_range=(0, 1))
model = LogisticRegression(max_iter=1000)

# Scale the features
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Train model
model.fit(X_train_scaled, y_train)

# Get predictions on validation set
y_pred_val = model.predict(X_val_scaled)
y_pred_proba_val = model.predict_proba(X_val_scaled)[:, 1]

# Calculate validation metrics
print("\nValidation Metrics:")
print(f"Accuracy: {accuracy_score(y_val, y_pred_val):.4f}")
print(f"Precision: {precision_score(y_val, y_pred_val):.4f}")
print(f"Recall: {recall_score(y_val, y_pred_val):.4f}")
print(f"F1: {f1_score(y_val, y_pred_val):.4f}")
print(f"AUC-ROC: {roc_auc_score(y_val, y_pred_proba_val):.4f}")

# Get predictions on test set
y_pred_test = model.predict(X_test_scaled)
y_pred_proba_test = model.predict_proba(X_test_scaled)[:, 1]

# Calculate test metrics
print("\nTest Metrics:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_test):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_test):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_test):.4f}")
print(f"F1: {f1_score(y_test, y_pred_test):.4f}")
print(f"AUC-ROC: {roc_auc_score(y_test, y_pred_proba_test):.4f}")

# Print feature indexes with the highest weights and their labels
feature_weights = model.coef_[0]
top_feature_indices = np.argsort(np.abs(feature_weights))[::-1][:10]  # Get top 10 feature indices
print("\nTop 10 Features with Highest Weights:")

# Create boolean mask for get_feature_labels_from_mask
mask = np.zeros(len(feature_weights), dtype=bool)
mask[top_feature_indices] = True

# Get labels and print table with weights
feature_labels = get_feature_labels_from_mask(mask)
for idx, label in zip(top_feature_indices, feature_labels):
    print(f"Feature {idx} ({label}): weight = {feature_weights[idx]:.4f}")



Validation Metrics:
Accuracy: 0.6817
Precision: 0.9637
Recall: 0.2387
F1: 0.3826
AUC-ROC: 0.6811

Test Metrics:
Accuracy: 0.6944
Precision: 0.9498
Recall: 0.2355
F1: 0.3774
AUC-ROC: 0.6744

Top 10 Features with Highest Weights:
+---------+----------------------------------------------+--------------+
|   Index | Label                                        |   Confidence |
+=========+==============================================+==============+
|      60 | Business and finance communication in Polish |           79 |
+---------+----------------------------------------------+--------------+
|      68 | Business and operational references          |           16 |
+---------+----------------------------------------------+--------------+
|      71 | Incomplete URL presence                      |           79 |
+---------+----------------------------------------------+--------------+
|      81 | Mentions Vikings and sports content          |           80 |
+---------+--------------------